In [1]:
from roboticstoolbox import DHRobot, RevoluteDH
import numpy as np
import time
import matplotlib.pyplot as plt
from spatialmath import SE3
from tqdm import tqdm

# Define DH parameters
dh = np.array([
    [0, -np.pi/2, 10, 0],
    [50, 0, 0, -np.pi/2],
    [0, -np.pi/2, 5, 0],
    [0, np.pi/2, 50, 0],
    [0, -np.pi/2, 0, 0],
    [0, 0, 10, np.pi]
])

# Create robot model
links = [RevoluteDH(d=row[2], a=row[0], alpha=row[1], offset=row[3]) for row in dh]
robot = DHRobot(links, name="MyRobot")

def calculate_single_pose(robot, joint_angles):
    """
    Calculate x, y, z position and roll, pitch, yaw angles for a single set of joint angles
    and measure the execution time.
    
    Args:
    robot (DHRobot): The robot model
    joint_angles (list or numpy.array): List of 6 joint angles in degrees
    
    Returns:
    dict: A dictionary containing the calculated pose and execution time
    """
    start_time = time.time()
    
    # Convert joint angles to radians
    q = np.deg2rad(joint_angles)
    
    # Calculate forward kinematics
    T = robot.fkine(q)
    
    # Extract position
    x, y, z = T.t
    
    # Extract Euler angles (ZYX convention: yaw, pitch, roll)
    yaw, pitch, roll = np.rad2deg(T.eul())
    
    end_time = time.time()
    execution_time = end_time - start_time
    
    return {
        'x': x,
        'y': y,
        'z': z,
        'roll': roll,
        'pitch': pitch,
        'yaw': yaw,
        'execution_time': execution_time
    }

In [2]:
# Example usage:
# Assuming you have a DHRobot object called 'robot'
joint_angles = [0, 45, -30, 60, -45, 90]  # Example joint angles in degrees
result = calculate_single_pose(robot, joint_angles)
print(f"Pose: x={result['x']:.4f}, y={result['y']:.4f}, z={result['z']:.4f}")
print(f"Orientation: roll={result['roll']:.4f}, pitch={result['pitch']:.4f}, yaw={result['yaw']:.4f}")
print(f"Execution time: {result['execution_time']:.6f} seconds")

Pose: x=91.3968, y=-1.1237, z=33.9993
Orientation: roll=147.9111, pitch=80.8805, yaw=-38.3316
Execution time: 0.000342 seconds


In [3]:
import random

# Assuming you have a DHRobot object called 'robot'

# Generate random joint angles
def random_joint_angles():
    return [random.uniform(-120, 120) for _ in range(6)]

# Test the function with a single set of joint angles
joint_angles = random_joint_angles()
result = calculate_single_pose(robot, joint_angles)

print("Single calculation:")
print(f"Joint angles: {joint_angles}")
print(f"Pose: x={result['x']:.4f}, y={result['y']:.4f}, z={result['z']:.4f}")
print(f"Orientation: roll={result['roll']:.4f}, pitch={result['pitch']:.4f}, yaw={result['yaw']:.4f}")
print(f"Execution time: {result['execution_time']:.6f} seconds")

# Test multiple calculations to get an average execution time
num_tests = 1000
total_time = 0

for _ in range(num_tests):
    joint_angles = random_joint_angles()
    result = calculate_single_pose(robot, joint_angles)
    total_time += result['execution_time']

average_time = total_time / num_tests

print(f"\nAverage execution time over {num_tests} calculations: {average_time:.6f} seconds")
print(f"Calculations per second: {1/average_time:.2f}")

Single calculation:
Joint angles: [-80.62061316945392, -53.12324607301727, 91.72867769880645, 6.767856140030702, 24.081054050497926, 43.290985426886664]
Pose: x=6.0078, y=-2.7405, z=-0.0558
Orientation: roll=54.7605, pitch=152.4103, yaw=-74.6612
Execution time: 0.000757 seconds

Average execution time over 1000 calculations: 0.000164 seconds
Calculations per second: 6103.92
